In [ ]:
import pandas as pd

df_creditcard = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/creditcard.csv")
df_loan=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/loan_applications.csv")
df_transaction=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/transactions.csv")

In [ ]:
df_creditcard.rename(columns={"Class": "fraud_flag"}, inplace=True)

df_creditcard["fraud_flag"].value_counts() #Porcentaje de transacciones NO fraudulentas y fraudulentas

,count
fraud_flag,
0,284315
1,492


In [ ]:
df_loan["fraud_flag"].value_counts()

,count
fraud_flag,
0,48974
1,1026


In [ ]:
df_transaction["fraud_flag"].value_counts()

,count
fraud_flag,
0,49498
1,502


In [ ]:
from sklearn.model_selection import train_test_split
df_credit_sample, _ = train_test_split(df_creditcard, train_size=50000, stratify=df_creditcard['Class'], random_state=42)
df_credit_sample["Class"].value_counts(normalize=True) * 100


,proportion
Class,
0,99.828
1,0.172


In [ ]:
# Calcular riesgo histórico promedio de fraude (proporción de fraudes)
riesgos_historicos = {
    'Tarjeta de Crédito': df_creditcard['fraud_flag'].mean(),
    'Préstamos': df_loan["fraud_flag"].mean(),
    'Transacciones': df_transaction["fraud_flag"].mean()
}
# Ordenar de mayor a menor riesgo
riesgos_ordenados = dict(sorted(riesgos_historicos.items(), key=lambda x: x[1], reverse=True))
# Mostrar resultados
for unidad, riesgo in riesgos_ordenados.items():
    print(f'{unidad}: {riesgo:.4%} de riesgo histórico')








Préstamos: 2.0520% de riesgo histórico
Transacciones: 1.0040% de riesgo histórico
Tarjeta de Crédito: 0.1727% de riesgo histórico


In [ ]:
df_creditcard["Amount"]
df_loan["loan_amount_requested"]
df_transaction["transaction_amount"]


In [ ]:
fraud_amount_creditcard = df_creditcard[df_creditcard['fraud_flag'] == 1]['Amount'].sum()
print(f'Monto total de transacciones fraudulentas en tarjetas de crédito: {fraud_amount_creditcard:,.2f}')

fraud_amount_loan= df_loan[df_loan['fraud_flag'] == 1]['loan_amount_requested'].sum()
print(f'Monto total de transacciones fraudulentas en préstamos: {fraud_amount_loan:,.2f}')

fraud_amount_transaction= df_transaction[df_transaction['fraud_flag'] == 1]['transaction_amount'].sum()
print(f'Monto total de transacciones fraudulentas en transacciones: {fraud_amount_transaction:,.2f}')






Monto total de transacciones fraudulentas en tarjetas de crédito: 60,127.97
Monto total de transacciones fraudulentas en préstamos: 515,731,000.00
Monto total de transacciones fraudulentas en transacciones: 4,144,916.47


In [ ]:
nofraud_amount_creditcard = df_creditcard[df_creditcard['fraud_flag'] == 0]['Amount'].sum()
print(f'Monto total de transacciones NO fraudulentas en tarjetas de crédito: {nofraud_amount_creditcard:,.2f}')

nofraud_amount_loan= df_loan[df_loan['fraud_flag'] == 0]['loan_amount_requested'].sum()
print(f'Monto total de transacciones NO fraudulentas en préstamos: {nofraud_amount_loan:,.2f}')

nofraud_amount_transaction= df_transaction[df_transaction['fraud_flag'] == 0]['transaction_amount'].sum()
print(f'Monto total de transacciones NO fraudulentas en transacciones: {nofraud_amount_transaction:,.2f}')

Monto total de transacciones NO fraudulentas en tarjetas de crédito: 25,102,462.04
Monto total de transacciones NO fraudulentas en préstamos: 25,179,921,000.00
Monto total de transacciones NO fraudulentas en transacciones: 246,138,800.00


In [ ]:
total_amount_creditcard = fraud_amount_creditcard + nofraud_amount_creditcard
total_amount_loan = fraud_amount_loan + nofraud_amount_loan
total_amount_transaction = fraud_amount_transaction + nofraud_amount_transaction
print(f"Monto total de transacciones en tarjetas de crédito: {total_amount_creditcard:,.2f}")
print(f"Monto total de transacciones en préstamos: {total_amount_loan:,.2f}")
print(f"Monto total de transacciones en transacciones: {total_amount_transaction:,.2f}")

Monto total de transacciones en tarjetas de crédito: 25,162,590.01
Monto total de transacciones en préstamos: 25,695,652,000.00
Monto total de transacciones en transacciones: 250,283,716.47


In [ ]:

print(f"Porcentaje de transacciones fraudulentas en tarjetas de crédito: {fraud_amount_creditcard/total_amount_creditcard:.2%}")
print(f"Porcentaje de transacciones fraudulentas en préstamos: {fraud_amount_loan/total_amount_loan:.2%}")
print(f"Porcentaje de transacciones fraudulentas en transacciones: {fraud_amount_transaction/total_amount_transaction:.2%}")

Porcentaje de transacciones fraudulentas en tarjetas de crédito: 0.24%
Porcentaje de transacciones fraudulentas en préstamos: 2.01%
Porcentaje de transacciones fraudulentas en transacciones: 1.66%


In [ ]:
Costo_promedio_fraude_TC =fraud_amount_creditcard/492 #CUser1
Costo_promedio_fraude_loan= fraud_amount_loan/1026 #CUser3
Costo_promedio_fraude_transaction= fraud_amount_transaction/502 #CUser2
print(f"Costo promedio de fraude en tarjetas de crédito: {Costo_promedio_fraude_TC}")
print(f"Costo promedio de fraude en préstamos: {Costo_promedio_fraude_loan}")
print(f"Costo promedio de fraude en transacciones: {Costo_promedio_fraude_transaction}")

Costo promedio de fraude en tarjetas de crédito: 122.21132113821139
Costo promedio de fraude en préstamos: 502661.79337231966
Costo promedio de fraude en transacciones: 8256.805723123816


In [ ]:
Baseline_prob = 0.08
C_promedio_total= Costo_promedio_fraude_TC + Costo_promedio_fraude_loan + Costo_promedio_fraude_transaction

In [ ]:
#Gtotal = 75988
Gtotal = 384807
# si le ofrecieramos lo mismo a todos los usuarios, partiendo desde el supuesto de que ambos datasets tuvieran la misma cantidad de usuarios (filas) con prob de fraude de 0.08, se estaría
#incurriendo en una pérdida de 15,732,166,490.72 de 25,971,098,306.48, es decir un 60.58%


Costo_total_fraude_sin_modelo = Baseline_prob * Gtotal * C_promedio_total
print(f"Costo total de fraude sin modelo: {Costo_total_fraude_sin_modelo:,.2f}")

Costo total de fraude sin modelo: 15,732,166,490.72


In [ ]:
Monto_total_movimiento = total_amount_creditcard + total_amount_loan + total_amount_transaction
print(f"Monto total de movimientos: {Monto_total_movimiento:,.2f}")

Monto total de movimientos: 25,971,098,306.48


In [ ]:
Usuarios_totales = 384807


In [ ]:
Porcentaje_costo_fraude = Costo_total_fraude_sin_modelo / Monto_total_movimiento
print(f"Porcentaje de costo de fraude: {Porcentaje_costo_fraude:.2%}")

Porcentaje de costo de fraude: 60.58%
